# General Imports

In [1]:
import os
import shutil
import re
import csv
import pandas as pd
from ase import io, atoms
from ase.build import surface, make_supercell, add_adsorbate, bulk
from ase.io import write, read
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import LBFGS

# .PWI File Generation (Grid)

#### Required Perameters

In [6]:
# Directory perameters
material = "SrCoO3_A.xyz"

# Cluster peram
cluster_name = "zayahcortright"
cluster_command = ""

# Define aborbate name(s)
options = ["O", "H", "OOH", "OH"]

In [7]:
# QE perameters

# Pseudopotentials
pseudopotentials = {
    'Ti': 'Ti.pbe-spn-kjpaw_psl.1.0.0.UPF', # 35 (280)
    "La": "La.paw.z_11.atompaw.wentzcovitch.v1.2.upf", # 40 (320)
    "Co": "Co_pbe_v1.2.uspp.F.UPF", # 45 (360)
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF", # 50 (400)
    "H": "H.pbe-rrkjus_psl.1.0.0.UPF", # 60 (480)
    "Sr": "Sr_pbe_v1.uspp.F.UPF" # 30 (240)
}

# Input data for QE .pwi files
input_data = {
    'system': {
        'ecutwfc': 60, 
        'ecutrho': 480,
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'degauss': 0.07,
    },
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'pwscf',
        'outdir': f'/bsuhome/{cluster_name}/scratch',
        'disk_io': 'low',
        'verbosity': 'high'
    },
    'electrons': {
        'conv_thr': 1e-8,
        'mixing_mode': 'local-TF',
        'electron_maxstep': 200,
        'mixing_beta': 0.15,
        #'diagonalization': 'rmm-davidson',
        'diagonalization': 'david',
    }
}

#### Generation

In [9]:
profile = EspressoProfile(
    command=f"{cluster_command}",
    pseudo_dir=f"/bsuhome/{cluster_name}/q-e/pseudo"
)

header_folder = material[:-4]  # Create the header folder based on the material name without .xyz
os.makedirs(header_folder, exist_ok=True)  # Ensure the header folder exists

save_dirs = []

read_material = io.read(material)

adsorbates = {
    "OOH": [("O", 3), ("O", 3 + 1.245956), ("H", 3 + 1.245956 + .97907)],
    "OH": [("O", 3), ("H", 3 + .97907)],
    "O": [("O", 3)],
    "H": [("H", 3)],
}

for abs_name in options:
    save_dir = os.path.join(header_folder, f"{material[:-4]}_{abs_name}")
    save_dirs.append(save_dir)
    if abs_name in adsorbates:
        for i in range(8):
            for j in range(8):
                prev = read_material.copy()
                for element, height in adsorbates[abs_name]:
                    add_adsorbate(read_material, element, height=height, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))

                bin_folder = "0" if i < 4 else "1"
                dir_name = f"{i}-{j}"
                file_name = f"{i}-{j}.xyz"
                file_path = os.path.join(save_dir, bin_folder, dir_name)
                os.makedirs(file_path, exist_ok=True)

                io.write(os.path.join(file_path, file_name), read_material)

                calc = Espresso(
                    profile=profile,
                    pseudopotentials=pseudopotentials,
                    input_data=input_data,
                    kpts=(4, 4, 1),
                    koffset=(0, 0, 0),
                )
                read_material.calc = calc
                io.espresso.write_espresso_in(
                    os.path.join(file_path, "espresso.pwi"),
                    read_material,
                    input_data=input_data,
                    pseudopotentials=pseudopotentials,
                    kpts=(4, 4, 1),
                    koffset=(0, 0, 0)
                )
                read_material = prev.copy()

print(f"Save this somewhere! {save_dirs}")


Save this somewhere! ['SrCoO3_A\\SrCoO3_A_O', 'SrCoO3_A\\SrCoO3_A_H', 'SrCoO3_A\\SrCoO3_A_OOH', 'SrCoO3_A\\SrCoO3_A_OH']


# Extract

In [10]:
# Function to extract energy from file
def extract_energy_from_file(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "!    total energy" in line:
                # Extract the energy value using regex
                energy = re.search(r"[-+]?\d*\.\d+|\d+", line)
                if energy:
                    return float(energy.group(0))
    return None

# Function to process folders
def process_folders(base_path, x_start, x_end):
    for i in range(x_start, x_end):
        for j in range(0, 8):
            folder_name = f"{i}-{j}"
            folder_path = os.path.join(base_path, folder_name)
            if os.path.isdir(folder_path):
                file_path = os.path.join(folder_path, "espresso.pwo")
                if os.path.isfile(file_path):
                    energy = extract_energy_from_file(file_path)
                    if energy is not None:
                        df.loc[len(df)] = [i, j, energy]

for folder in save_dirs:
    # Initialize the dataframe
    df = pd.DataFrame(columns=["x", "y", "SCF"])

    # Process folders in "0"
    process_folders(os.path.join(folder, "0"), 0, 4)

    # Process folders in "1"
    process_folders(os.path.join(folder, "1"), 4, 8)

    # Save the dataframe to a CSV file
    df.to_csv(f"{folder}_OUT.csv", index=False)